# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `10`
* Students: `Fluhr Hugo, Donzier Paul, de Goumoëns Frédéric, Cionca Alexandre`
* Dataset: `TMDb, tmdb_5000_movies.csv and tmdb_5000_credits.csv`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [2]:
# DATA LOADING

# We load both datasets, credits contains the casts of the movies and movies contains movie features
credits=pd.read_csv('../../dataset/TMDb/tmdb_5000_credits.csv')
movies=pd.read_csv('../../dataset/TMDb/tmdb_5000_movies.csv')
print(type(credits.cast[0]))
# The columns cast and crew of the dataset credits are strings, we use json.loads to convert into dict
json_columns = ['cast']
for column in json_columns:
     credits[column] = credits[column].apply(json.loads)

json_columns = ['keywords']
for column in json_columns:
     movies[column] = movies[column].apply(json.loads)

<class 'str'>


In [3]:
# DATA FILTERING

# Here we fuse the 2 datasets into one and drop all columns useless for this milestone
movie_df=credits.join(movies, rsuffix='other')

In [4]:
#movie_df=movie_df.drop(columns=['budget','overview','original_language','vote_average','crew',
#                            'popularity','homepage','id','keywords','original_title',
#                            'production_companies','production_countries','release_date',
#                            'revenue','runtime','spoken_languages','tagline','titleother'])

# To reduce the dataset, we filter unreleased movies and movies with a vote count inferior to 100
movie_df=movie_df[movie_df.status=='Released']
movie_df=movie_df[movie_df.vote_count >=100]
movie_df=movie_df[movie_df.original_language == 'en']
#movie_df=movie_df.drop(columns=['vote_count','status'])

# Reseting the index after filtering
movie_df=movie_df.reset_index(drop=True)

In [5]:
# Creating empty column which will be filled with the id of the actors playing in each movie
movie_df['actors_id']=""
movie_df['keywords_key']=""

# Extraction of actors ID from the cast dictionnary and adding them to the added column
for i in range(len(movie_df.title)):
    strs=[]
    for ii in range(len(movie_df.cast[i])):
        strs.append(movie_df.cast[i][ii].get('id'))
    movie_df.at[i,'actors_id']=strs
    
    for ii in range(len(movie_df.keywords[i])):
        strs.append(movie_df.keywords[i][ii].get('name'))
    movie_df.at[i,'keywords_key']=strs

#The "cast" column is no longer needed
#movie_df = movie_df.drop(columns=['cast'])

# Preview of the Milestone 1 dataframe
print(type(movie_df.actors_id[0]))

<class 'list'>


### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [6]:
# Number of nodes in the network
n_nodes = len(movie_df)

# The adjacency matrix contains the number of common actors for each pair of nodes (movies)
# The matrix is set to be symmetric since the relation of shared actor is undirected
adjacency=np.zeros((n_nodes,n_nodes), dtype=float)

for i in range(len(movie_df.index)):
    for ii in range(i+1,(len(movie_df.index))):
            adjacency[i,ii]=adjacency[ii,i]=len(set(movie_df.keywords_key[i]).intersection(set(movie_df.keywords_key[ii])))

print(adjacency.max())

# We normalize by the maximum number of shared actors between two movies found in the dataset
adjacency = np.divide(adjacency,adjacency.max())
adjacency_2 = np.divide(adjacency,10/42)
adjacency_2[adjacency_2 > 1] = 1

42.0


In [7]:
adjacency_2[adjacency_2 < 0.2] = 0
adjacency[:5,:5]

array([[0.        , 0.        , 0.        , 0.        , 0.07142857],
       [0.        , 0.        , 0.02380952, 0.02380952, 0.        ],
       [0.        , 0.02380952, 0.        , 0.        , 0.02380952],
       [0.        , 0.02380952, 0.        , 0.        , 0.        ],
       [0.07142857, 0.        , 0.02380952, 0.        , 0.        ]])

In [8]:
# We see from the adjacency matrix that there are some movies that don't have any connections to any other movies,
# these nodes don't present any interest to us so we remove them from both the dataframe and the adjacency matrix
i=0
while i < len(adjacency):
    if not any(adjacency[i]):
        adjacency = np.delete(adjacency, i, 0)
        adjacency = np.delete(adjacency, i, 1)
        movie_df=movie_df.drop([i])
        movie_df=movie_df.reset_index(drop=True)
    else:
        i+=1
        
# We need to update n_nodes since we have dropped some movies
n_nodes=len(adjacency)
n_nodes

3018

In [9]:
# We see from the adjacency matrix that there are some movies that don't have any connections to any other movies,
# these nodes don't present any interest to us so we remove them from both the dataframe and the adjacency matrix
i=0
while i < len(adjacency_2):
    if not any(adjacency_2[i]):
        adjacency_2 = np.delete(adjacency_2, i, 0)
        adjacency_2 = np.delete(adjacency_2, i, 1)
        movie_df=movie_df.drop([i])
        movie_df=movie_df.reset_index(drop=True)
    else:
        i+=1
        
# We need to update n_nodes since we have dropped some movies
n_nodes_2=len(adjacency_2)
n_nodes_2

2970

In [10]:
#movie_df=movie_df.drop(columns=[''])
movie_df.to_csv('saved_features_ext.csv', encoding='utf-8')
#movie_df

## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
# Dots represent connections between movies regardless of their weight
plt.figure(figsize=(10,10))
plt.spy(adjacency, markersize=0.1)
plt.title('adjacency matrix')

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
# We find in the first lecture of the course the expression of L_max, the maximum number of links for a fully connected graph.
# L max is equal to the number of distinct pair of nodes present in the graph
L_max=int(n_nodes*(n_nodes-1)/2)
print('L_max = ',L_max)

# To compute L, we need to count how many non-zero elements there are in the top-right half of the adjacency matrix
L=0
for i in range(adjacency.shape[0]):
    for ii in range(i,adjacency.shape[1]):
        L+=(adjacency[i][ii]!=0)
print('L =',L)

print('Density:', L/L_max)

In [ ]:
# We find in the first lecture of the course the expression of L_max, the maximum number of links for a fully connected graph.
# L max is equal to the number of distinct pair of nodes present in the graph
L_max=int(n_nodes_2*(n_nodes_2-1)/2)
print('L_max = ',L_max)

# To compute L, we need to count how many non-zero elements there are in the top-right half of the adjacency matrix
L=0
for i in range(adjacency_2.shape[0]):
    for ii in range(i,adjacency_2.shape[1]):
        L+=(adjacency_2[i][ii]!=0)
print('L =',L)

print('Density:', L/L_max)

$L_{max}$ for our graph which has 3138 nodes is equal to 4'921'953

In our graph, there are 163'621 links

We find a density: $L/L_{max}$ = 0.033, showing that our graph, much like many real graphs, is sparse.

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

Our graph is undirected. The links between movies represent the number of actors they share which is an undirected relation.

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
# Saving of both dataframe and adjacency matrix
movie_df.to_csv('saved_features.csv', sep='\t', encoding='utf-8')
np.save('saved_adjacency',adjacency)
np.save('saved_adjacency_2',adjacency_2)
movie_df.title[225]

### Question 4

Are the edges of your graph weighted?

Yes, the edges are weighted, the weight is proportional to the number of shared actors.
Since we have normalized the weights, they represent a proportion of how many actors two movies share divided by the maximum of shared actors between two movies found in the dataset.

### Question 5

What is the degree distibution of your network? 

In [ ]:
# We create an array containing the degree of each node
degree = np.sum(adjacency,0)
degree_2 = np.sum(adjacency_2,0)

# Here we verify that there are as many elements in the degree array as there are nodes (movies) in the graph
assert len(degree) == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
weights_2 = np.ones_like(degree_2) / float(n_nodes)
plt.subplot(211)
plt.hist(degree, weights=weights, bins=30);
plt.subplot(212)
plt.hist(degree_2,weights=weights_2, bins=30);

What is the average degree?

In [ ]:
average_deg=np.divide(np.sum(degree),n_nodes)
print(average_deg)

### Question 6

Comment on the degree distribution of your network.

We see from the degree distribution histogram that our graph follows a Poisson degree distribution. Unlike most real networks that follow a power law, our network has fewer weakly connected nodes. This is in part due to the fact that we discarded nodes with a degree of zero when filtering our data.

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
# We implement a version of the BFS algorithm in an additional function that we will need for both questions 7 and 8

def bfs(adjacency, source_node):
    # Explored is an array where explored[a_certain_node]= 0 when this a_certain_node cannot be reached by source node
    # or = 1 when this a_certain_node can be reached
    n_nodes=len(adjacency)
    explored=np.full(n_nodes,0)
    # Store nodes to explore in a list
    to_explore = [source_node]
    
    # The source node is already explored
    explored[source_node] = 1  
    
    # Stay in loop while there are still nodes to explore
    while to_explore:
       # Pop first node frome the queue and add it to to the explored nodes
        current_node = to_explore.pop(0)
        explored[current_node] = 1
        # Copy the line of the matrix corresponding to the current node to see its connections
        neighbours = np.argwhere(adjacency[current_node]).flatten()
        
        # Set the neighbours of current_node to be explored
        for i in range(len(neighbours)):
            if not explored[neighbours[i]]:
                to_explore.append(neighbours[i])
                explored[neighbours[i]]=1
    
    return explored

In [ ]:
def connected_graph(adjacency):
    """Determines whether a graph is connected.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """
    
    # We call the previously implemented bfs function and check if the number of reached nodes starting from the
    # first node is equal to the number of nodes in the graph
    n_nodes=len(adjacency)
    connected = (len(np.argwhere(bfs(adjacency,0)))==n_nodes)
    
    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
connected_graph(adjacency)

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
# Declaring a function that removes both column and row full of 0s from a matrix (numpy array)
# This function is used in find_components()
def no_zero(matrix):
    
    i=0
    while i < len(matrix):
        if not any(matrix[i]):
            matrix = np.delete(matrix, i, 0)
            matrix = np.delete(matrix, i, 1)
        else:
            i+=1
    
    
    return matrix

In [ ]:
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """

    n_nodes=len(adjacency)
    components=[]
    # We declare an array of size n_nodes which contains for each node its component index,
    # e.g. to which component it belongs
    comp_idx=np.full(n_nodes,0)
    component_label=1
    
    # The implemented bfs() function returns an array of size (n_nodes,1) which contains a one if the node
    # is reachable from source and a zero if it isn't. This is the indicator vector of the connected
    # component containing the first node (in our case). Each iteration of the while loop labels all nodes
    # of the i-th connected component with i.
    while len(np.argwhere(comp_idx))<n_nodes:
        comp_idx +=component_label*bfs(adjacency,np.argwhere(comp_idx==0)[0,0])
        component_label+=1
    
    # We now extract one adjacency matrix per component and store them in the "components" list
    for i in range(1,component_label):
        matrix=adjacency@np.diag(comp_idx == i)
        matrix=no_zero(matrix)
        components.append(matrix)
    
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [ ]:
comp = find_components(adjacency_2)
size = 0

print('Number of connected components:',len(comp), 'component(s)')

# Finding the largest connected component
for mat in comp:
    if len(mat) > size:
        biggest = mat
        size = len(biggest)

print('Size of the largest connected component:',len(biggest), 'node(s)')

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [11]:
def compute_shortest_path_lengths(adjacency, source):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """

    # Firstly, we transform the matrix to an unweighted one
    adjacency=np.where(adjacency != 0, 1, 0)
    n_nodes=adjacency.shape[0]
    
    # All shortest path lengths are initialy set to infinity
    shortest_path_lengths=np.full(n_nodes,np.inf)
    
    # Distance from source node to itself is 0
    shortest_path_lengths[source]=0
    # Creation of a list containing the index of every unvisited_node
    unvis_nodes=list(range(n_nodes))
    
    # We stay in the loop as long as there are nodes that haven't been visited
    while unvis_nodes:
        dist_min=np.inf
        current_node=None
        
        # Search for the unvisited node with the shortest shortest_path_length and set it as current_node
        for i in range(len(unvis_nodes)):
            if (shortest_path_lengths[unvis_nodes[i]]<dist_min): 
                dist_min=shortest_path_lengths[unvis_nodes[i]]
                current_node=unvis_nodes[i]
        
        # Remove the current_node from the unvisited_nodes
        unvis_nodes.remove(current_node)
        
        # We take the neighbours of current_node that are still unvisited
        unvis_neighbours=np.intersect1d(np.argwhere(adjacency[current_node]),np.asarray(unvis_nodes))
        
        # Check for each unvisited_nodes if there is an alt_path going through current_node shorter
        # than the previously stored shortest_path_length (spl), if there is, update the spl with alt_path
        for i in range(len(unvis_neighbours)):
            alt_path=shortest_path_lengths[current_node]+1
            if (shortest_path_lengths[unvis_neighbours[i]]>alt_path):
                shortest_path_lengths[unvis_neighbours[i]]=alt_path
    
    # Casting into a list of int (we previously used floats to be able to have np.inf)
    shortest_path_lengths=shortest_path_lengths.tolist()
    shortest_path_lengths = [int(el) for el in shortest_path_lengths]
                
    return shortest_path_lengths

In [ ]:
# Printing the first 20 shortest paths from the first node (movie)
print(compute_shortest_path_lengths(adjacency,0)[:20])

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
# Reducing adjacency matrix size to first 1000 nodes and taking the largest connected component of this reduced matrix
red_comp = find_components(adjacency[:1000,:1000])
size = 0

for mat in red_comp:
    if (len(mat)>size):
        red_mat = mat

In [ ]:
diameter=0
# Finding the longest shortest path in the graph
for i in range(len(red_mat)):
    diameter=max(max(compute_shortest_path_lengths(red_mat,i)),diameter)
print(diameter)

### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    
    # Once again, taking the unweighted adjacency matrix
    adj_bin=np.where(adjacency != 0, 1, 0)
    pow_adj=adj_bin
    
    # Compute adj_bin to the power length
    if length==1:
        n_paths=adj_bin[source,target]
    else:    
        for i in range(length-1):
            pow_adj=np.matmul(pow_adj,adj_bin)
        n_paths=pow_adj[source,target]   
    
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
# We create an unweighted version ot the adjacency matrix
unw_adj=np.where(adjacency != 0, 1, 0)

# Sum all elements of unw_adjacency^3

adj_cub=np.matmul(np.matmul(unw_adj,unw_adj),unw_adj)

print(np.sum(np.sum(np.int64(adj_cub))))

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    # Number of links between the neighbours of node on which we want to calculate the clustering coeff.
    Li = 0
    # List of neighbours of node on which we want to calculate the clustering coeff.
    ki_list = []

    for i in range(len(adjacency)):
        # Find neighboring nodes
        if (adjacency[node,i] != 0 and i != node):
            # Store neighbouring nodes
            ki_list.append(i)
            
    # If the number of neighbours is equal to or less than 1, the clustering coefficient is 0 :       
    if (len(ki_list))<=1:
        return 0.0
    else :
        # For every neighbour
        for i in range(len(ki_list)):
            for j in ki_list:
                # Count every links between the neighbours only once
                if ((adjacency[ki_list[i],j]) and (j in ki_list[i:])):
                    # No difference in the execution when giving unweighted or weighted adjacency matrix
                    Li += 1
        # Formula given in lecture
        clustering_coefficient = (2*Li)/(len(ki_list)*(len(ki_list) - 1))
    
    return clustering_coefficient

### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
average_clustering_coeff = 0.
for i in range(n_nodes):
    average_clustering_coeff += compute_clustering_coefficient(adjacency, i)/n_nodes
print(average_clustering_coeff)